In [1]:

from network import transformNet
from modules import EncoderNet,DecoderNet,DiscriminatorNet_reconstruction,GeneratorNet,DiscriminatorNet_feature
from functional import noise
#from train import train,eval_generate
import resnet


import invresnet
import matplotlib.pylab as plt

from dataload import load_data ,batchfy 
from torchsummary import summary
import torch
from torch import nn,optim

SyntaxError: invalid syntax (transformNet.py, line 74)

In [2]:


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type(torch.cuda.FloatTensor)

NameError: name 'torch' is not defined

In [ ]:
train_load,test_load=batchfy(batch_size=64)